In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.stats import norm
import collections
import warnings
import Filtering
import Tool

In [8]:
# Construct a general load profile of users from non-EV users

data_non = pd.read_csv("data/user_2335.csv")
profile = data_non[["Month", "Day", "Hour", "Minute", "consumption"]]
Non_EVcustomer_id = [
    2361,
    2818,
    3039,
    3456,
    3538,
    4031,
    5746,
    7536,
    7800,
    7901,
    7951,
    8386,
    8565,
    9019,
    9160,
    9278,
    9922,
]
non_EVload = np.zeros((35040, len(Non_EVcustomer_id)))

for i, iid in enumerate(Non_EVcustomer_id):
    data_non = pd.read_csv(f"data/user_{iid}.csv")
    data_non = data_non[["Month", "Day", "Hour", "Minute", "consumption"]]
    non_EVload[:, i] = data_non["consumption"]

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    profile["consumption"] = np.nanmean(non_EVload, axis=1)

profile = profile[["Month", "Day", "Hour", "Minute", "consumption"]]
Non_EVProfile = profile.groupby(["Month", "Hour", "Minute"], as_index=False).mean()
Non_EVProfile.rename(columns={"consumption": "consumption_NonEV"}, inplace=True)

In [9]:
for customer_id in [4373, 1642, 8156, 6139, 7719]:
    Rate_EST = 3
    alpha = 0.1
    percent = 0.1

    # Filter out AC load
    data = pd.read_csv(f"data/user_{customer_id}.csv")
    acfilter = Filtering.ACfilter()
    helper = Tool.Disaggregation_tool()
    data = acfilter.ACfiltering(customer_id, data)

    # Find out base load of customers in each month
    data_base = data[["Month", "consumption"]]
    base_load = data_base.groupby(["Month"], as_index=False).min()
    base_load.rename(columns={"consumption": "base_load"}, inplace=True)
    data = pd.merge(data, base_load, on=["Month"])
    data_NonEV = Non_EVProfile.copy()

    dic = dict()
    for i in range(len(base_load)):
        dic[(base_load.iloc[i]["Month"])] = base_load.iloc[i]["base_load"]

    # Initialize the non-charging periods
    data["under"] = data.apply(
        lambda x: 1
        if (
            x["consumption"] < x["base_load"] + Rate_EST
            or x["forward"] == 0
            or x["backward"] == 0
        )
        else 0,
        axis=1,
    )

    # Initialize the regular load profile (constrcuted from the given EV user and the general load profile constructed above)
    data_NEV = data[data["under"] == 1]
    data_group = data_NEV.groupby(by=["Month", "Hour", "Minute"], as_index=False).mean()
    ratio = sum(data_NonEV["consumption_NonEV"]) / sum(data_group["consumption"])
    data_NonEV["consumption_NonEV"] = data_NonEV["consumption_NonEV"] / ratio
    data_NonEV = data_NonEV[["Month", "Hour", "Minute", "consumption_NonEV"]]
    data_group = pd.merge(
        data_group, data_NonEV, how="right", on=["Month", "Hour", "Minute"]
    )
    data_group = data_group.fillna(0)

    regular_profile = []
    for i in range(len(data_group)):
        if not data_group.iloc[i, :]["consumption"]:
            regular_profile.append(data_group.iloc[i, :]["consumption_NonEV"])
        else:
            regular_profile.append(
                (1 - alpha) * data_group.iloc[i, :]["consumption"]
                + alpha * data_group.iloc[i, :]["consumption_NonEV"]
            )

    data_group["regular_profile"] = regular_profile
    data_group = data_group[["Month", "Hour", "Minute", "regular_profile"]]
    data_total = pd.merge(data, data_group, on=["Month", "Hour", "Minute"])
    data_total = data_total.sort_values(by=["Month", "Day", "Hour", "Minute"])
    data_total["prev1"] = data_total["consumption"].shift(1)
    data_total["dif1"] = data_total["consumption"] - data_total["prev1"]
    data_total["prev2"] = data_total["consumption"].shift(2)
    data_total["dif2"] = data_total["consumption"] - data_total["prev2"]
    data_total.to_csv(f"data/EVuser_{customer_id}_profile.csv", index=False)

    epochs = 1
    for i in range(epochs):

        # Read users' load profile from previous epoch
        data = pd.read_csv(f"data/EVuser_{customer_id}_profile.csv")

        ##################
        data = data.dropna()
        ##################

        # Identify the potential start and end of charging periods
        data["higher"] = data.apply(
            lambda x: helper.charging_status(x, Rate_EST, 1 / 2), axis=1
        )
        data["start"] = data.apply(
            lambda x: helper.change_point_status_start(x, Rate_EST, 2 / 3), axis=1
        )
        data["end"] = data.apply(
            lambda x: helper.change_point_status_end(x, Rate_EST, 2 / 3), axis=1
        )

        start_candidate = []
        candidate = []

        for i in range(len(data)):
            if (
                data.iloc[i]["start"] == 1
                and data.iloc[i]["higher"] == 1
                and data.iloc[i]["forward"] * data.iloc[i]["backward"]
            ):
                start_candidate.append(i)

        # Identify the (start,end) pair of a potential charging period
        visited = set()
        start_visited = set()
        for start in start_candidate:
            for end in range(start + 1, len(data)):
                if (
                    data.iloc[end]["end"] == -1 and data.iloc[end]["higher"] <= 0
                ) and start not in start_visited:
                    candidate.append([start, end])
                    start_visited.add(start)
                    for i in range(start, end + 1):
                        visited.add(i)
                    break
                elif end - start > 24:
                    break

        start_visited = set()
        for start in start_candidate:
            for end in range(start + 1, len(data)):
                if (
                    data.iloc[end]["end"] == -1
                    and data.iloc[end + 1]["end"] == -1
                    and data.iloc[end + 2]["end"] == -1
                    and start not in start_visited
                ):
                    candidate.append([start, end])
                    start_visited.add(start)
                    for i in range(start, end + 1):
                        visited.add(i)
                    break
                elif end - start > 24:
                    break

        # Delete detected consecutive periods (most likely they are not true charging periods)
        candidate = helper.consecutive_filter(candidate)

        EVcharging = []
        for start, end in candidate:
            mean, std = helper.distribution(data, start, end, visited)
            if helper.ChargingDetection(data, start, end, mean, std, Rate_EST, dic):
                EVcharging.append([start, end - 1])

        Chargingrate = []
        for start, end in EVcharging:
            Chargingrate.append(max(data.iloc[start]["dif2"], data.iloc[start]["dif1"]))

        lower = np.percentile(Chargingrate, 25)
        upper = np.percentile(Chargingrate, 75)
        Rate_EST = np.mean([i for i in Chargingrate if lower < i < upper])

        Charging_period = set()
        bound = set()
        for start, end in EVcharging:
            for k in range(start, end + 1):
                Charging_period.add(k)
            bound.add(start - 1)
            bound.add(end + 1)

        data["estimation"] = helper.estimation(
            Rate_EST, len(data), Charging_period, bound
        )

        Non_EVPeriods = data[data["estimation"] < 1]
        Non_EVPeriods = Non_EVPeriods[["Month", "Day", "Hour", "Minute", "consumption"]]
        Non_EVPeriods.to_csv(f"data/EVuser_{customer_id}_Updated.csv", index=False)

        P = len([i for i in data["car"] if i > 0.5])
        N = len([i for i in data["car"] if i < 0.5])
        print(f"customer_id:{customer_id}, Estimated chraging rate:{np.round(Rate_EST, 2)}")
        ######################UPDATE#####################

        # Update users' regular load profile
        Non_EVPeriods = pd.read_csv(f"data/EVuser_{customer_id}_Updated.csv")
        user = pd.read_csv(f"data/user_{customer_id}.csv")

        user = user[
            ["Month", "Day", "Hour", "Minute", "consumption", "car", "grid", "solar"]
        ]
        user = pd.merge(user, base_load, how="left", on=["Month"])

        data_group = Non_EVPeriods.groupby(
            by=["Month", "Hour", "Minute"], as_index=False
        ).mean()
        data_group.rename(columns={"consumption": "regular_profile"}, inplace=True)
        data_group = data_group[["Month", "Hour", "Minute", "regular_profile"]]
        data_group = pd.merge(
            user, data_group, how="left", on=["Month", "Hour", "Minute"]
        )
        data_group = pd.merge(
            data_group, data_NonEV, how="right", on=["Month", "Hour", "Minute"]
        )
        data_group = data_group.fillna(0)

        regular_profile = []

        for i in range(len(data_group)):
            if not data_group.iloc[i, :]["regular_profile"]:
                regular_profile.append(data_group.iloc[i, :]["consumption_NonEV"])
            else:
                regular_profile.append(
                    (1 - percent) * data_group.iloc[i, :]["regular_profile"]
                    + percent * data_group.iloc[i, :]["consumption_NonEV"]
                )
        data_group["regular_profile"] = regular_profile

        data_group = data_group.sort_values(by=["Month", "Day", "Hour", "Minute"])
        data_group["prev1"] = data_group["consumption"].shift(1)
        data_group["dif1"] = data_group["consumption"] - data_group["prev1"]
        data_group["prev2"] = data_group["consumption"].shift(2)
        data_group["dif2"] = data_group["consumption"] - data_group["prev2"]
        data_group = data_group.sort_values(by=["Month", "Day", "Hour", "Minute"])

        for_back = pd.read_csv(f"data/forward_{customer_id}.csv")
        for_back = for_back[["Month", "Day", "Hour", "Minute", "forward", "backward"]]
        data_group = pd.merge(
            data_group, for_back, on=["Month", "Day", "Hour", "Minute"]
        )

        data_group.to_csv(
            f"data/EVuser_{customer_id}_profile.csv",
            index=False,
        )

    os.remove(f"data/EVuser_{customer_id}_profile.csv")
    os.remove(f"data/EVuser_{customer_id}_Updated.csv")
    os.remove(f"data/forward_{customer_id}.csv")

customer_id:4373, Estimated chraging rate:3.27
customer_id:1642, Estimated chraging rate:3.36
customer_id:8156, Estimated chraging rate:3.3
customer_id:6139, Estimated chraging rate:3.25
customer_id:7719, Estimated chraging rate:3.22


In [10]:
np.percentile(data["temperature"], 80)

28.85

In [11]:
np.percentile(data["temperature"], 20)

11.0